Working with data from <code>Prepocessing/data/spikeTrains</code> should have folder for each experment setup (in terms of disctance of magnent in mm). Each folder should have a list of spike trains as csv files. Spike trains are recorded as events per 0.05 secs. The index file has the list of each spike train path and the class they are in. Classes are configureable via the preprocessing but should be somthing like this 

<table style="border:none;padding: 10px;margin: auto;">
    <tr style="border:none;padding: 10px;margin: auto;">
        <td style="border:none;padding: 10px;margin: auto;">
        9/5mm
        <table>
            <th>Class</th><th>Number of reactions</th>
            <tr> <td>High</td> <td>>300</td> </tr>
            <tr> <td>Medium</td> <td>300-230</td> </tr>
            <tr> <td>Low</td> <td>&lt;230</td> </tr>
        </table>
        </td>
        <td style="border:none;padding: 10px;margin: auto;">
        3mm
        <table>
            <th>Class</th><th>Number of reactions</th>
            <tr> <td>High</td> <td>>150</td> </tr>
            <tr> <td>Medium</td> <td>100-150</td> </tr>
            <tr> <td>Low</td> <td>&lt;100</td> </tr>
        </table>
        </td>
        <td style="border:none;padding: 10px;margin: auto;">
        2/1.5mm
        <table>
            <th>Class</th><th>Number of reactions</th>
            <tr> <td>High</td> <td>>50</td> </tr>
            <tr> <td>Medium</td> <td>30-50</td> </tr>
            <tr> <td>Low</td> <td>&lt;30</td> </tr>
        </table>
        </td>
    </tr>
</table>

We are using Intel's lava-nc framework https://lava-nc.org

In [26]:
%reset
# progress_bar for when stuff takes a while to load
def progress_bar(current, total, bar_length=20):
    fraction = current / total

    arrow = int(fraction * bar_length - 1) * '-' + '>'
    padding = int(bar_length - len(arrow)) * ' '

    ending = '\n' if current == total else '\r'

    print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

----
#### DataLoader

This is our data loader. Should get an index file for an experment type (9mm,5mm etc). The index file should contain a path to each indiviual training experiment and the class it belongs to high, medium, or low.
Note that the class is represented as a int, the map is shown below

|class         | numerical value|
|--------------|----------------|
|<i>high</i>   |               2|
|<i>medium</i> |               1|
|<i>low</i>    |               0|

Example index file (Prepocessing/data/spikeTrains/1.5-SpikeTrains/index.csv)
<pre>
spikeTrain_1.csv,0
spikeTrain_2.csv,0
spikeTrain_3.csv,1
spikeTrain_4.csv,0
spikeTrain_5.csv,2
spikeTrain_6.csv,1
</pre>


Example spik train file (Prepocessing/data/spikeTrains/1.5-SpikeTrains/spikeTrain_1.csv)
<pre>
0
0
0
.
.
.
0
0
1
0
.
.
.
</pre>

In [27]:
import numpy as np
import pandas as pd
import os

class expermentDataloader:
    def __init__(
        self,
        index_file: str, 
        data_path: str,
    ):
        self.root_dir = data_path
        self.expermentSikeTrainsIndex = pd.read_csv(index_file) # self.landmarks_frame = pd.read_csv(csv_file)
        self.spikeTrains = [
            f"{os.path.join(self.expermentSikeTrainsIndex.iloc[i, 0])}" for i in range(len(self.expermentSikeTrainsIndex)) 
        ]
        self.expermentClasses = self.expermentSikeTrainsIndex.iloc[:, 1]

    def __getitem__(self, index):
        CSVlines = pd.read_csv(os.path.join(self.root_dir,self.spikeTrains[index])).to_numpy()
        eventClass = self.expermentClasses[index]
        return np.array(list(CSVlines.flatten())), int(eventClass)

    def __len__(self):
        return len(self.expermentSikeTrainsIndex)

In [28]:
indexFile5mm = "./Prepocessing/data/spikeTrains/5-SpikeTrains/index.csv"
PathTo5mmSpikeTrains = "./Prepocessing/data/spikeTrains/5-SpikeTrains"

trainingData = expermentDataloader(indexFile5mm,PathTo5mmSpikeTrains)

print(f"Is NOT all zeros: {np.any(trainingData[0][0])}")
print(len(trainingData[0][0]))
print(f"Number of spikes: {sum(trainingData[0][0])}")
trainingData[0]


Is NOT all zeros: True
2999
Number of spikes: 5


(array([0, 0, 0, ..., 0, 0, 0]), 0)

In [29]:
print(f"Is NOT all zeros: {np.any(trainingData[1][0])}")
print(len(trainingData[1][0]))
print(f"Number of spikes: {sum(trainingData[1][0])}")
trainingData[1]

Is NOT all zeros: True
2999
Number of spikes: 32


(array([0, 0, 0, ..., 0, 0, 0]), 1)

In [30]:
print(f"Is NOT all zeros: {np.any(trainingData[len(trainingData)-1][0])}")
print(len(trainingData[len(trainingData)-1][0]))
print(f"Number of spikes: {sum(trainingData[len(trainingData)-1][0])}")
trainingData[len(trainingData)-1]

Is NOT all zeros: True
2999
Number of spikes: 51


(array([0, 0, 0, ..., 0, 0, 0]), 2)

In [31]:
steps_per_spike_train = len(trainingData[len(trainingData)-1][0])
steps_per_spike_train

2999

----
# Training STDP Model

----
# backprop

So it looks like lava-dl is what we want. This is a library that allows you to use lava-nc as a deep NN. There is a tutorial <a href=https://github.com/lava-nc/lava-dl/blob/a4fd4fc2ecd0da89e4aa0e9516533e85d4fbbcb0/tutorials/lava/lib/dl/slayer/nmnist/train.ipynb>here</a> that seems to be close to what we want.

There are two ways to use the lib
1. lava.lib.dl.slayer for natively training Deep Event-Based Networks.
2. lava.lib.dl.bootstrap for training rate coded SNNs.

We are going to try the first way, natively training with slayer

In [32]:
import os, sys
import glob
import zipfile
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

class lava.lib.dl.slayer.block.cuba.Dense(args, kwargs)

Bases: AbstractCuba, AbstractDense

CUBA LIF dense block class. The block is 8 bit quantization ready.

Parameters

* neuron_params (dict, optional) – a dictionary of CUBA LIF neuron parameter. Defaults to None.

* in_neurons (int) – number of input neurons.

* out_neurons (int) – number of output neurons.

* weight_scale (int, optional) – weight initialization scaling. Defaults to 1.

* weight_norm (bool, optional) – flag to enable weight normalization. Defaults to False.

* pre_hook_fx (optional) – a function pointer or lambda that is applied to synaptic weights before synaptic operation. None means no transformation. Defaults to None.

* delay (bool, optional) – flag to enable axonal delay. Defaults to False.

* delay_shift (bool, optional) – flag to simulate spike propagation delay from one layer to next. Defaults to True.

* mask (bool array, optional) – boolean synapse mask that only enables relevant synapses. None means no masking is applied. Defaults to None.

* count_log (bool, optional) – flag to return event count log. If True, an additional value of average event rate is returned. Defaults to False.

training: bool

In [33]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        neuron_params = {
                'threshold'     : 1.25,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,     
            }
        neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        
        self.blocks = torch.nn.ModuleList([
                slayer.block.cuba.Dense(neuron_params_drop, 1, 512, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params_drop, 512, 512, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, 512, 10, weight_norm=True),
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike
    
    def grad_flow(self, path):
        # helps monitor the gradient flow
        grad = [b.synapse.grad_norm for b in self.blocks if hasattr(b, 'synapse')]

        plt.figure()
        plt.semilogy(grad)
        plt.savefig(path + 'gradFlow.png')
        plt.close()

        return grad

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

In [34]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [35]:
trained_folder = 'Trained'
os.makedirs(trained_folder, exist_ok=True)


net = Network().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)


train_loader = DataLoader(dataset=trainingData, batch_size=32, shuffle=True)

There are a couple loss functions we can use 
* `SpikeTime`: precise spike time based loss when target spike train is known.
* `SpikeRate`: spike rate based loss when desired rate of the output neuron is known.
* `SpikeMax`: negative log likelihood losses for classification without any rate tuning.

We will use SpikeRate for our simple classification problem

In [36]:
error = slayer.loss.SpikeRate(true_rate=0.2, false_rate=0.03, reduction='sum').to(device)

Setup the untils for training

In [37]:
stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(net, error, optimizer, stats, classifier=slayer.classifier.Rate.predict)

Okay lets try the training loop

In [38]:
epochs = 100

for epoch in range(epochs):
    for i, (input, label) in enumerate(train_loader): # training loop
        output = assistant.train(input, label)
    print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')
        
    # for i, (input, label) in enumerate(test_loader): # training loop
    #     output = assistant.test(input, label)
    # print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')
        
    if epoch%20 == 19: # cleanup display
        print('\r', ' '*len(f'\r[Epoch {epoch:2d}/{epochs}] {stats}'))
        stats_str = str(stats).replace("| ", "\n")
        print(f'[Epoch {epoch:2d}/{epochs}]\n{stats_str}')
    
    if stats.testing.best_accuracy:
        torch.save(net.state_dict(), trained_folder + '/network.pt')
    stats.update()
    stats.save(trained_folder + '/')
    net.grad_flow(trained_folder + '/')

RuntimeError: Expected 4D (unbatched) or 5D (batched) input to conv3d, but got input of size: [32, 2999]

----
# Truing LAVA dataloader

In [19]:
import numpy as np
import pandas as pd
import os

class expermentDataloader:
    def __init__(
        self,
        index_file: str, 
        data_path: str,
    ):
        self.root_dir = data_path
        self.expermentSikeTrainsIndex = pd.read_csv(index_file) # self.landmarks_frame = pd.read_csv(csv_file)
        self.spikeTrains = [
            f"{os.path.join(self.expermentSikeTrainsIndex.iloc[i, 0])}" for i in range(len(self.expermentSikeTrainsIndex)) 
        ]
        self.expermentClasses = self.expermentSikeTrainsIndex.iloc[:, 1]

    def __getitem__(self, index):
        CSVlines = pd.read_csv(os.path.join(self.root_dir,self.spikeTrains[index])).to_numpy()
        eventClass = self.expermentClasses[index]
        return np.array(list(CSVlines.flatten())), int(eventClass)

    def __index__(self, index):
        return self.__getitem__(index)

    def __len__(self):
        return len(self.expermentSikeTrainsIndex)

In [20]:
indexFile5mm = "./Prepocessing/data/spikeTrains/5-SpikeTrains/index.csv"
PathTo5mmSpikeTrains = "./Prepocessing/data/spikeTrains/5-SpikeTrains"

trainingData = expermentDataloader(indexFile5mm,PathTo5mmSpikeTrains)

print(f"Is NOT all zeros: {np.any(trainingData[0][0])}")
print(len(trainingData[0][0]))
print(f"Number of spikes: {sum(trainingData[0][0])}")
trainingData[0]


Is NOT all zeros: True
2999
Number of spikes: 5


(array([0, 0, 0, ..., 0, 0, 0]), 0)

In [21]:
# general imports
import typing as ty

# Import premade Processes
from lava.proc.dense.process import Dense
from lava.proc.lif.process import LIF

# Import Processes and Process level primitives
from lava.magma.core.process.process import AbstractProcess
from lava.magma.core.process.variable import Var
from lava.magma.core.process.ports.ports import  OutPort, InPort

# Import ProcessModels and ProcessModels level primitives
from lava.magma.core.decorator import implements, requires
from lava.magma.core.model.py.model import PyLoihiProcessModel
from lava.magma.core.model.py.ports import PyOutPort, PyInPort
from lava.magma.core.model.py.type import LavaPyType
from lava.magma.core.resources import CPU

# Import execution protocol and hardware resources
from lava.magma.core.sync.protocols.loihi_protocol import LoihiProtocol
from lava.magma.core.run_configs import Loihi1SimCfg
from lava.magma.core.run_conditions import RunSteps, RunContinuous

# Import Monitor and graph stuff
from lava.proc.monitor.process import Monitor
import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt

In [22]:
# replace with '''lava.proc.io.dataloader.SpikeDataloader''' see https://lava-nc.org/lava/lava.proc.io.html
from lava.proc.io.dataloader import SpikeDataloader
steps_per_spike_train = len(trainingData[len(trainingData)-1][0])
input = SpikeDataloader(dataset=trainingData,interval=steps_per_spike_train)

In [23]:
input.vars.member_names

['data', 'interval', 'offset']

In [24]:
number_of_spike_trains = 3
output = Monitor()
groundTruth = Monitor()
output.probe(input.s_out, steps_per_spike_train*number_of_spike_trains)
groundTruth.probe(input.ground_truth, steps_per_spike_train*number_of_spike_trains)

In [25]:
number_of_spike_trains = 1

for i in range(number_of_spike_trains):
    print(f"Running Spike Train {i}")
    run_cfg = Loihi1SimCfg(select_tag="floating_pt")
    run_condition = RunSteps(num_steps=steps_per_spike_train)
    input.run(condition=run_condition, run_cfg=run_cfg)

Running Spike Train 0
Encountered Fatal Exception: slice indices must be integers or None or have an __index__ method
Traceback: Encountered Fatal Exception: slice indices must be integers or None or have an __index__ method

Traceback (most recent call last):
  File "/home/khood/anaconda3/envs/dna/lib/python3.10/site-packages/lava/magma/runtime/runtime.py", line 95, in target_fn
    actor.start(*args, **kwargs)
  File "/home/khood/anaconda3/envs/dna/lib/python3.10/site-packages/lava/magma/core/model/py/model.py", line 83, in start
    p.start()
  File "/home/khood/anaconda3/envs/dna/lib/python3.10/site-packages/lava/magma/core/model/interfaces.py", line 34, in start
    csp_port.start()
  File "/home/khood/anaconda3/envs/dna/lib/python3.10/site-packages/lava/magma/compiler/channels/pypychannel.py", line 82, in start
    self._array = [
  File "/home/khood/anaconda3/envs/dna/lib/python3.10/site-packages/lava/magma/compiler/channels/pypychannel.py", line 86, in <listcomp>
    buffer=sel

KeyboardInterrupt: 